In [ ]:
%%capture
%pip install accelerate peft bitsandbytes transformers trl
!pip install pyngrok
!pip install flask-cors
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import threading
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

app = Flask(__name__)
CORS(app)
base_model = "NousResearch/Llama-2-7b-chat-hf"

dataset1 = "heliosbrahma/mental_health_chatbot_dataset"

new_model = "checkpoint-150"
logging.set_verbosity(logging.CRITICAL)
pipe = pipeline(task="text-generation", model=new_model, tokenizer=new_model, max_length=150)
@app.route('/api/message', methods=['POST'])
def process_message():
    data = request.json
    user_input = data.get('input', '')
    result = pipe(f"<s>[INST] {user_input} [/INST]")
    generated_text = result[0]['generated_text']
    return jsonify({'response': generated_text})

def run_flask():
    app.run(host='0.0.0.0', port=5003)

threading.Thread(target=run_flask).start()

!cloudflared tunnel --url http://localhost:5003 --no-autoupdate


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

 * Serving Flask app '__main__'
 * Debug mode: off
2024-05-28T11:29:17Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2024-05-28T11:29:17Z INF Requesting new quick Tunnel on trycloudflare.com...


Address already in use
Port 5003 is in use by another program. Either identify and stop that program, or start the server with a different port.


2024-05-28T11:29:17Z INF +--------------------------------------------------------------------------------------------+
2024-05-28T11:29:17Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2024-05-28T11:29:17Z INF |  https://throughout-accessibility-wider-jazz.trycloudflare.com                             |
2024-05-28T11:29:17Z INF +--------------------------------------------------------------------------------------------+
2024-05-28T11:29:17Z INF Cannot determine default configuration path. No file [config.yml config.yaml] in [~/.cloudflared ~/.cloudflare-warp ~/cloudflare-warp /etc/cloudflared /usr/local/etc/cloudflared]
2024-05-28T11:29:17Z INF Version 2024.5.0
2024-05-28T11:29:17Z INF GOOS: linux, GOVersion: go1.22.2, GoArch: amd64
2024-05-28T11:29:17Z INF Settings: map[ha-connections:1 no-autoupdate:true protocol:quic url:http://localhost:5003]
2024-05-28T11:29:17Z INF cloudflared will not automatically update if installed by a 

INFO:werkzeug:127.0.0.1 - - [28/May/2024 11:44:46] "OPTIONS /api/message HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/May/2024 11:45:40] "POST /api/message HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/May/2024 11:46:23] "OPTIONS /api/message HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/May/2024 11:47:18] "POST /api/message HTTP/1.1" 200 -


2024-05-28T12:18:11Z INF Initiating graceful shutdown due to signal interrupt ...
2024-05-28T12:18:12Z INF Unregistered tunnel connection connIndex=0 event=0 ip=198.41.200.113
2024-05-28T12:18:12Z ERR Failed to serve quic connection error="Application error 0x0 (remote)" connIndex=0 event=0 ip=198.41.200.113
2024-05-28T12:18:12Z ERR Serve tunnel error error="Application error 0x0 (remote)" connIndex=0 event=0 ip=198.41.200.113
2024-05-28T12:18:12Z INF Retrying connection in up to 1s connIndex=0 event=0 ip=198.41.200.113
2024-05-28T12:18:12Z ERR no more connections active and exiting
2024-05-28T12:18:12Z INF Tunnel server stopped
2024-05-28T12:18:12Z ERR icmp router terminated error="context canceled"
2024-05-28T12:18:12Z INF Metrics server stopped
